In [ ]:
teams_info = ("https://www.iplt20.com/teams")

In [ ]:
from bs4 import BeautifulSoup
import requests

import pandas as pd


In [ ]:
# Helper Functions

def check_response(url):
    return requests.get(url).status_code



In [ ]:
resp = requests.get(teams_info)
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
def get_teams_information(url):
    try:
        resp = requests.get(url)
        # resp.raise_for_status()
        print("GOT GREEN LIGHT FROM URL!!!")
    except requests.exceptions.RequestException as e:
        print("Request Faild : {e}")

    soup = BeautifulSoup(resp.text, "html.parser")
    
    l = []  # Will hold the infomation that is scrapped

    for i in soup.find_all("a", {"data-team_name":True}):
        team_name = i['data-team_name']  
        
        url = i['href']

        abbrevation = i.find('span').text

        trophy_div = i.find('div', class_='trophy-text-align')
        trophy_year =  trophy_div.text if trophy_div else None

        logo = i.find("img")['src']

        l.append([team_name, abbrevation, trophy_year, logo, url])

    df = pd.DataFrame(l, columns = ["team_name","abbrevation","trophy_year","logo","team_url"])

    return df

In [ ]:
df = get_teams_information(teams_info)

In [ ]:
df

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)
df = pd.read_csv("scrapped_teams_info/ipl_teams_data.csv")

mi_url = "https://www.iplt20.com/teams/mumbai-indians"
srh_url = "https://www.iplt20.com/teams/sunrisers-hyderabad"

In [ ]:
list(df['team_url'])

In [ ]:

resp = requests.get(mi_url)
soup = BeautifulSoup(resp.text, "html.parser")

In [ ]:
soup

In [ ]:
def get_team_players(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        print("GOT GREEN LIGHT FROM URL!!!")
    except requests.exceptions.RequestException as e:
        print(f"Request Failed: {e}")
        return pd.DataFrame()  # Return empty DataFrame on failure

    soup = BeautifulSoup(resp.text, "html.parser")
    players = []

    

    for i in soup.find_all("a", {"data-player_name": True}):
        # Head shot png
        head_shot_tag = i.find(class_='lazyload')
        head_shot_png = head_shot_tag.get('data-src') if head_shot_tag else None

        # Player name
        player_name = i['data-player_name']

        # Player type
        player_type_tag = i.find(class_='d-block w-100 text-center')
        player_type = player_type_tag.text.strip() if player_type_tag else None

        # Scrapping team's icon
        teams_icon_div = i.find(class_='teams-icon')
        tag = teams_icon_div.find_all("img") if teams_icon_div else []

        # Scrapping individual's icons
        player_type_icon = captain_icon = foreign_player_icon = None

        if len(tag) == 1:
            player_type_icon = tag[-1]['src']
        elif len(tag) == 2:
            if "captain" in tag[0]['src']:
                captain_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
            else:
                foreign_player_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
        elif len(tag) >= 3:
            captain_icon = tag[0]['src']
            foreign_player_icon = tag[1]['src']
            player_type_icon = tag[2]['src']

        players.append([
            player_name,
            head_shot_png,
            player_type,
            player_type_icon,
            captain_icon,
            foreign_player_icon
        ])

    df = pd.DataFrame(players, columns=['player_name', 'head_shot_png', 'player_type','player_type_icon', 'captain_icon', 'foreign_player_icon'])
    return df


In [ ]:
get_team_players(mi_url)

In [ ]:
get_team_players(srh_url)


In [ ]:
# get_team_players(mi_url).to_csv("test.csv")

In [ ]:
test1 = soup.find_all("a",{"data-player_name" : True})[5]

In [ ]:
test1['data-player_name']

In [ ]:
test1.find(class_ = 'd-block w-100 text-center').text

In [ ]:
resp = requests.get(srh_url)
soup = BeautifulSoup(resp.text, "html.parser")

for i in soup.find_all("a",{"data-player_name" : True}):
    print(i['data-player_name'])
    tag = i.find(class_ = 'teams-icon').find_all("img")
    print(tag)
    if len(tag) == 1:
            player_type_icon = tag[-1]['src']
            captain_icon = None
            foreign_player_icon = None

            print(player_type_icon,captain_icon,foreign_player_icon)
    elif len(tag) == 2:
        if "captain" in tag[-2]['src']:
            player_type_icon = tag[-1]['src']
            captain_icon = tag[-2]['src']
            foreign_player_icon = None
            print(player_type_icon,captain_icon,foreign_player_icon)

        else:
            player_type_icon = tag[-1]['src']
            captain_icon = None
            foreign_player_icon = tag[-2]['src']
            print(player_type_icon,captain_icon,foreign_player_icon)

    else:
        player_type_icon = tag[-1]['src']
        foreign_player_icon = tag[-2]['src']
        captain_icon = tag[-3]['src']
        print(player_type_icon,captain_icon,foreign_player_icon)


In [ ]:
def get_team_players(url):
    try:
        resp = requests.get(url)
        resp.raise_for_status()
        print(f"GOT GREEN LIGHT FROM {url}!!!")
    except requests.exceptions.RequestException as e:
        print(f"Request Failed: {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(resp.text, "html.parser")
    players = []

    team_name = url.rstrip("/").split("/")[-1]

    for i in soup.find_all("a", {"data-player_name": True}):
        head_shot_tag = i.find(class_='lazyload')
        head_shot_png = head_shot_tag.get('data-src') if head_shot_tag else None

        player_name = i['data-player_name']

        player_type_tag = i.find(class_='d-block w-100 text-center')
        player_type = player_type_tag.text.strip() if player_type_tag else None

        teams_icon_div = i.find(class_='teams-icon')
        tag = teams_icon_div.find_all("img") if teams_icon_div else []

        player_type_icon = captain_icon = foreign_player_icon = None
        if len(tag) == 1:
            player_type_icon = tag[-1]['src']
        elif len(tag) == 2:
            if "captain" in tag[0]['src']:
                captain_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
            else:
                foreign_player_icon = tag[0]['src']
                player_type_icon = tag[1]['src']
        elif len(tag) >= 3:
            captain_icon = tag[0]['src']
            foreign_player_icon = tag[1]['src']
            player_type_icon = tag[2]['src']

        players.append([
            player_name,
            head_shot_png,
            player_type,
            player_type_icon,
            captain_icon,
            foreign_player_icon
        ])

    df = pd.DataFrame(players, columns=[
        'player_name', 'head_shot_png', 'player_type',
        'player_type_icon', 'captain_icon', 'foreign_player_icon'
    ])
    df['team_name'] = team_name  
    return df


In [ ]:
df = merge_data_for_team_players(team_urls)

In [ ]:
df['team_name'].value_counts()

In [ ]:
df

In [ ]:
df = pd.read_csv("scrapped_teams_info/ipl_teams_data.csv")


In [ ]:
df

In [ ]:
resp = requests.get("https://www.iplt20.com/match/2025/1839")

In [ ]:
soup = BeautifulSoup(resp.text)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()

driver.get("https://www.iplt20.com/match/2025/1839")
driver.find_elements(By.CLASS_NAME, "cmdEvent fourCmd mcBall mcBall mcBall mcBall mcBall mcBall")
time.sleep(10)
driver.close()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome()
driver.get("http://www.python.org")
assert "Python" in driver.title
elem = driver.find_element(By.NAME, "q")
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source
driver.close()

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
kkr = driver.page_source

In [ ]:
soup = BeautifulSoup(driver.page_source)

In [ ]:
soup

In [ ]:
soup.find(class_="matchOrder mob-hide ng-binding ng-scope").text

In [ ]:
soup.find(class_="ms-matchdate ng-binding").text

In [ ]:
soup.find(class_="matGround ng-binding").text

In [ ]:
soup.find_all(class_="cmdOver mcBall bgFour mcBall mcBall mcBall mcBall mcBall")[0].text

In [ ]:
print(ball_wrapper[1].find(class_ = "commentaryText ng-binding").text)

In [ ]:
# commentary
ball_wrapper[0].find(class_ = "commentaryText ng-binding").text

In [ ]:
ball_wrapper[0]

In [ ]:
ball_wrapper = soup.find_all(class_ = "ballWrapper ng-scope")[1:]
l = []
for i in ball_wrapper:
    commentary_tag = i.find(class_="commentaryText ng-binding")
    commentry = commentary_tag.text if commentary_tag else None

    over_tag = i.find("p", class_="cmdOver")
    over_and_runs = over_tag.text.strip() if over_tag else None

    l.append([commentry,over_and_runs])

    import pandas as pd

# Sample DataFrame
df = pd.DataFrame(l, columns=['commentary', 'over_and_run'])

# Drop rows where 'over_and_run' is NaN
df = df[df['over_and_run'].notna()]

# Split the string and ensure it's a list
df['over_and_run'] = df['over_and_run'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Now extract elements safely
df['over'] = df['over_and_run'].apply(lambda x: x[0] if len(x) > 0 else None)
df['run'] = df['over_and_run'].apply(lambda x: x[1] if len(x) > 1 else None)
df['extras'] = df['over_and_run'].apply(lambda x: x[2] if len(x) > 2 else None)



In [ ]:
import pandas as pd
df = pd.DataFrame(l, columns = ['commentary','over_and_run'])

# Seperate out the over, runs and wickets
df = df[df['over_and_run'].notna()]
df["over_and_run"] = df['over_and_run'].apply(lambda x : x.split(' '))

In [ ]:
df

In [ ]:
df['run'] = df['over_and_run'].apply(lambda x : x[1])

In [ ]:
df[df['run'] != 'W']['run'].astype("int").sum()

In [ ]:
df[df['run'] == 'W'].shape[0]

In [ ]:
df['extras'].unique()

In [ ]:
df.info()

In [ ]:
ball_wrapper[0].find("p", class_="cmdOver mcBall bgFour mcBall mcBall mcBall mcBall mcBall").text

In [ ]:
tabs = driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')

In [ ]:
tabs

In [ ]:
for tab in tabs:
    print(tab.text)

In [ ]:
for tab in tabs:
    if "KKR" in tab.text:
        tab.click()
        time.sleep(5)
        


In [ ]:
kkr_innings = driver.page_source
kkr_soup = BeautifulSoup(kkr_innings)

In [ ]:
ball_wrapper = kkr_soup.find_all(class_ = "ballWrapper ng-scope")[1:]
l = []
for i in ball_wrapper:
    commentary_tag = i.find(class_="commentaryText ng-binding")
    commentry = commentary_tag.text if commentary_tag else None

    over_tag = i.find("p", class_="cmdOver")
    over_and_runs = over_tag.text.strip() if over_tag else None

    l.append([commentry,over_and_runs])

    import pandas as pd

# Sample DataFrame
df = pd.DataFrame(l, columns=['commentary', 'over_and_run'])

# Drop rows where 'over_and_run' is NaN
df = df[df['over_and_run'].notna()]

# Split the string and ensure it's a list
df['over_and_run'] = df['over_and_run'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Now extract elements safely
df['over'] = df['over_and_run'].apply(lambda x: x[0] if len(x) > 0 else None)
df['run'] = df['over_and_run'].apply(lambda x: x[1] if len(x) > 1 else None)
df['extras'] = df['over_and_run'].apply(lambda x: x[2] if len(x) > 2 else None)



In [ ]:
df

In [ ]:
df['extras'].value_counts()

In [ ]:
str(df[df['run'] != 'W']['run'].astype("int").sum()) + '/' +str( df[df['run'] == 'W']['run'].shape[0])

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
driver = webdriver.Chrome()


url = "https://www.iplt20.com/match/2025/1799"
driver.get(url)

In [ ]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')


In [ ]:
for i in driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]'):
    print(i.text)

In [ ]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[11].text

In [ ]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[11].click()

In [ ]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[11]
first_inning_soup = driver.page_source


In [ ]:
from bs4 import BeautifulSoup

In [ ]:
first_innging = BeautifulSoup(first_inning_soup)

In [ ]:
driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')[12].click()

In [ ]:
first_innging.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope')[0].text

In [ ]:
driver.quit()

In [ ]:
for i in first_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope'):
    print(i.text)


In [ ]:
for i in second_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope'):
    print(i.text)


In [ ]:
driver = webdriver.Chrome()
driver.get(url)
tag_to_switch_innings = driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')
first_inning = tag_to_switch_innings[11]

In [ ]:
tag_to_switch_innings[11].text

In [ ]:
ball_wrapper = first_soup.find_all(class_ = "ballWrapper ng-scope")[1:]
l = []
for i in ball_wrapper:
    commentary_tag = i.find(class_="commentaryText ng-binding")
    commentry = commentary_tag.text if commentary_tag else None

    over_tag = i.find("p", class_="cmdOver")
    over_and_runs = over_tag.text.strip() if over_tag else None

    l.append([commentry,over_and_runs])

    import pandas as pd

# Sample DataFrame
df = pd.DataFrame(l, columns=['commentary', 'over_and_run'])

# Drop rows where 'over_and_run' is NaN
df = df[df['over_and_run'].notna()]

# Split the string and ensure it's a list
df['over_and_run'] = df['over_and_run'].apply(lambda x: x.split() if isinstance(x, str) else [])

# Now extract elements safely
df['over'] = df['over_and_run'].apply(lambda x: x[0] if len(x) > 0 else None)
df['run'] = df['over_and_run'].apply(lambda x: x[1] if len(x) > 1 else None)
df['extras'] = df['over_and_run'].apply(lambda x: x[2] if len(x) > 2 else None)



In [ ]:
driver.quit()

In [ ]:
for i in kkr_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope'):
    print(i.text)

In [ ]:
kkr_soup.find_all(class_ = 'ap-inner-tb-click ng-binding ng-scope')

In [ ]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

import pandas as pd

In [ ]:
def get_match_soups(url):
    
    driver = webdriver.Chrome()
    driver.get(url)
    tag_to_switch_innings = driver.find_elements(By.XPATH, '//a[@href="javascript:void(0);"]')
    first_inning = tag_to_switch_innings[11]
    whos_first_inning = first_inning.text
    first_inning.click()
    time.sleep(3)
    first_inning_soup = BeautifulSoup(driver.page_source, "html.parser")
    second_inning = tag_to_switch_innings[12]
    whos_second_inning = second_inning.text
    second_inning.click()
    time.sleep(3)
    second_inning_soup = BeautifulSoup(driver.page_source, "html.parser")

    driver.quit()

    return first_inning_soup,second_inning_soup, whos_first_inning, whos_second_inning

In [ ]:
first_inning_soup,second_inning_soup, whos_first_inning, whos_second_inning = get_match_soups("https://www.iplt20.com/match/2025/1799")

In [ ]:
def get_match_metadata(soup):
    """
    The function provides metadata of the match up between two teams.
    Input : Provide the soup of the innings data
    Ouput : Gives out a array of [Match Number, Venue, Date, Time]
    """
    l = []
    match_number = soup.find(class_ = "matchOrder mob-hide ng-binding ng-scope").text 
    l.append(match_number)
    
    for i in soup.find(class_ = "ap-match-place col-100 floatLft textCenter re ng-scope").find_all('span'):
        l.append(i.text)

    return l

In [ ]:
get_match_metadata(first_inning_soup)

In [ ]:
def get_ball_by_ball_commentary(soup):
    # ball_wrapper = soup.find_all('div', class_ = ['ballWrapper ng-scope','cmdOver mcBall mcBall mcBall mcBall mcBall mcBall'])
    ball_wrapper = soup.select('div.ballWrapper.ng-scope')
    ball_wrapper = ball_wrapper[1:]
    
    # To Store the scrapped data
    l = []

    for i in ball_wrapper:
        if i.p.contents[0].strip() != '':
            ball_number = i.p.contents[0].strip()
            event_on_that_ball = i.find('i').text.strip()

            # Extract bowler vs batter info
            bowling_info_tag = i.find('div', class_='commentaryStartText')
            bowling_info = bowling_info_tag.get_text(strip=True) if bowling_info_tag else None

            # Extract detailed commentary
            commentary_tag = i.find('div', class_='commentaryText')
            commentary = commentary_tag.get_text(strip=True) if commentary_tag else None

            l.append({
                'ball': ball_number,
                'event': event_on_that_ball,
                'bowling_info': bowling_info,
                'commentary': commentary,
            })

    df = pd.DataFrame(l)
    return df
    

    

In [ ]:
get_ball_by_ball_commentary(second_inning_soup)

In [ ]:
l = []
for i in get_ball_by_ball_commentary(second_inning_soup):
    if i.p.contents[0].strip() != '':
        ball_number = i.p.contents[0]
        event_on_that_ball = i.find('i').text
        

In [ ]:
l = []

for i in get_ball_by_ball_commentary(second_inning_soup):
    if i.p.contents[0].strip() != '':
        ball_number = i.p.contents[0].strip()
        event_on_that_ball = i.find('i').text.strip()

        # Extract bowler vs batter info
        bowling_info_tag = i.find('div', class_='commentaryStartText')
        bowling_info = bowling_info_tag.get_text(strip=True) if bowling_info_tag else None

        # Extract detailed commentary
        commentary_tag = i.find('div', class_='commentaryText')
        commentary = commentary_tag.get_text(strip=True) if commentary_tag else None

        l.append({
            'ball': ball_number,
            'event': event_on_that_ball,
            'bowling_info': bowling_info,
            'commentary': commentary
            'Innings' : 
        })


In [ ]:
df = pd.DataFrame(l)

In [ ]:
df

In [ ]:
df['event'].value_counts()

In [ ]:
df[df['ball'] == '15.3']

In [ ]:
print(df[df['ball'] == '15.3']['commentary'].iloc[0])

In [ ]:
print(df[df['ball'] == '11.4']['commentary'].iloc[0])


In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")

In [ ]:
len(get_ball_by_ball_commentary(first_inning_soup)[0])

In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[10].text

In [ ]:
len(get_ball_by_ball_commentary(second_inning_soup))

In [ ]:
get_ball_by_ball_commentary(second_inning_soup)[1].text


In [ ]:
for i in get_ball_by_ball_commentary(second_inning_soup):
    print(i.text)


In [ ]:
for i in get_ball_by_ball_commentary(first_inning_soup):
    i.select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


In [ ]:
l = []
for i in get_ball_by_ball_commentary(first_inning_soup):
    l.append(i.select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n"))


In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText div.commentaryText")

In [ ]:
commentary_div = get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText div.commentaryText")
commentary_text = commentary_div.get_text(strip=True)
print(commentary_text)


In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[0].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")

In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[1].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


In [ ]:
get_ball_by_ball_commentary(first_inning_soup)[2].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


In [ ]:
test = get_ball_by_ball_commentary(first_inning_soup)[-5].select_one("div.cmdText").get_text(separator="\n", strip=True).split("\n")


In [ ]:
for i in test:

    print(i)